# Project Outline

## Introduction

Molecular dynamics simulations is a method that applies equations of motion for each specified particle in a defined system to directly simulate dynamics. Molecular dynamics in the field of chemistry has allowed scientists to observe the dynamics of a system in microscopic detail which is usually not possible with traditional methods of chemical analysis. The use of MD has been applied to a variety of chemical reactions and molecules. It is used to model chemical processes and even used as an alternative method to carry out chemical reactions. This has allowed a great reduction in the labor, waste, and resources used in traditional lab settings. 

This project will take advantage of this method to simulate molecules of water in a system that is spherically shaped. Once the system is set up, we will simulate the system heating up and observe how that affects the system. We will simulate it so that the the molecules go through all 3 of its phases. We will be comparing the density as a funtion of temperature, radius and time to the properties of water and see if the simulation behaves as water does in reality. 